In [1]:
from sqlalchemy import create_engine, inspect
import pandas as pd
import geopandas as gpd
import plotly.express as px
import matplotlib.pyplot as plt
import os
os.chdir('/Users/tanyatsui/Documents/01_Projects/housingEmissions')

from data_processing._common.query_runner import QueryRunner

# define database connection
db_name = "urbanmining"
username = "postgres"
password = "Tunacompany5694!"
host = "localhost"  # or your host if it's not local
port = "5432"  # default port for PostgreSQL

connection_string = f"postgresql://{username}:{password}@{host}:{port}/{db_name}"
engine = create_engine(connection_string)

In [29]:
def get_df_byStatus(table_name, municipality_name='Groningen'): 
    # get embodied emissions for housing in NL
    query = f''' 
    SELECT year, status, SUM(emissions_embodied_kg) AS emissions_embodied_kg, SUM(sqm) AS sqm
    FROM {table_name} -- emissions_embodied_housing_nl 
    WHERE year >= 2012 AND year <= 2021 -- AND municipality = '{municipality_name}'
    GROUP BY year, status
    ORDER BY year, status
    '''
    df_byStatus = pd.read_sql_query(query, engine)
    
    def translate_status(x): 
        if x == 'Bouw gestart': 
            return 'Construction'
        elif x == 'Pand gesloopt': 
            return 'Demolition'
        elif x in ['renovation - post2020', 'renovation - pre2020', 'renovation - s1', 
                   'transformation - adding units', 'transformation - function change']: 
            return 'Renovation'
        else: 
            return x 
    df_byStatus.status = df_byStatus.status.map(lambda x: translate_status(x))
    df_byStatus['emissions_embodied_tons'] = df_byStatus.emissions_embodied_kg / 1000
    return df_byStatus

def plot_embodied_emissions(df_byStatus): 
    # bar chart for embodied emissions per year
    fig = px.bar(df_byStatus, x='year', y='emissions_embodied_tons', color = 'status', title='Embodied emissions per year for housing in NL', 
                 labels={'emissions_embodied_tons': 'Embodied emissions (tCO2eq)', 'year': 'Year', 'status': 'Emission Type'})
    fig.update_layout(width=800)
    fig.show()

df = get_df_byStatus('emissions_embodied_housing_nl')
df['scenario'] = 's0 - baseline'
df_s1 = get_df_byStatus('emissions_embodied_housing_nl_s1')
df_s1['scenario'] = 's1 - circular economy'
df_all = pd.concat([df, df_s1])
df_all

fig = px.bar(df_all, 
             x='year', 
             y='emissions_embodied_tons', 
             color='status', 
             barmode='stack', 
             facet_col='scenario',
             labels={'emissions_embodied_tons': 'Emissions Embodied (Tons)', 'year': 'Year', 'status': 'Status'},
             title='Emissions Embodied in Tons by Year and Scenario')
fig.show()

In [32]:
df

,year,status,emissions_embodied_kg,sqm,emissions_embodied_tons,scenario
0,2012,Construction,1.551191e+09,4908833.0,1551191.228,s0 - baseline
1,2012,Demolition,4.654752e+08,6045132.0,465475.164,s0 - baseline
2,2012,Renovation,7.401169e+08,5873944.0,740116.944,s0 - baseline
3,2012,Renovation,1.733487e+08,1375783.0,173348.658,s0 - baseline
4,2012,Renovation,7.569979e+07,600792.0,75699.792,s0 - baseline
5,2013,Construction,1.138968e+09,3604329.0,1138967.964,s0 - baseline
6,2013,Demolition,3.950053e+08,5129939.0,395005.303,s0 - baseline
7,2013,Renovation,6.921189e+08,5493007.0,692118.882,s0 - baseline
8,2013,Renovation,2.474044e+08,1963527.0,247404.402,s0 - baseline
9,2013,Renovation,1.129845e+08,896702.0,112984.452,s0 - baseline


In [3]:
df = df_all.groupby(['year', 'scenario']).emissions_embodied_tons.sum().reset_index()
fig = px.bar(df, 
             x='year', 
             y='emissions_embodied_tons', 
             color='scenario', 
             barmode='group', 
             labels={'emissions_embodied_tons': 'Emissions Embodied (Tons)', 'year': 'Year', 'status': 'Status'},
             title='Emissions Embodied in Tons by Year and Scenario')
fig.update_layout(width=1000)
fig.show()

In [6]:
df_scenario = df_all.groupby(['scenario', 'status']).emissions_embodied_tons.sum().reset_index()

fig = px.bar(
    df_scenario, x='scenario', y='emissions_embodied_tons', color='status', barmode='stack', 
    title='Embodied emissions per scenario for housing in NL (2012-2021)'
    )
fig.update_layout(width=650)
fig.show()

In [23]:
df_scenario_emissions_total = df_scenario.groupby('scenario').emissions_embodied_tons.sum().reset_index()
emissions_s0 = df_scenario_emissions_total[df_scenario_emissions_total.scenario == 's0 - baseline'].emissions_embodied_tons.values[0]
emissions_s1 = df_scenario_emissions_total[df_scenario_emissions_total.scenario == 's1 - circular economy'].emissions_embodied_tons.values[0]
difference = emissions_s0 - emissions_s1
print(f's1 reduced embodied emissions by {round(difference/1000000, 2)} million tons ({round(difference/emissions_s0*100)}%)')

s1 reduced embodied emissions by 1.52 million tons (5%)


In [13]:
df_scenario_emissions_total

,scenario,emissions_embodied_tons
0,s0 - baseline,3.127917e+07
1,s1 - circular economy,2.976110e+07


In [5]:
df_scenario = df_all.groupby(['scenario', 'status']).emissions_embodied_tons.sum().reset_index()
df_scenario
fig = px.bar(
    df_scenario, x='status', y='emissions_embodied_tons', color='scenario', barmode='group', 
    title='Embodied emissions per scenario for housing in NL (2012-2021)'
    )
fig.update_layout(width=650)
fig.show()

In [4]:
# get embodied emissions for housing in NL
query = ''' 
SELECT year, status, SUM(emissions_embodied_kg) AS emissions_embodied_kg
FROM emissions_embodied_housing_nl_s1
WHERE year >= 2012 AND year <= 2021
GROUP BY year, status
ORDER BY year, status
'''
df_byStatus = pd.read_sql_query(query, engine)
def translate_status(x): 
    if x == 'Bouw gestart': 
        return 'Construction'
    elif x == 'Pand gesloopt': 
        return 'Demolition'
    elif x in ['renovation - post2020', 'renovation - pre2020', 'renovation - s1', 
               'transformation - adding units', 'transformation - function change']: 
        return 'Renovation'
    else: 
        return x 
df_byStatus.status = df_byStatus.status.map(lambda x: translate_status(x))
df_byStatus['emissions_embodied_tons'] = df_byStatus.emissions_embodied_kg / 1000

# bar chart for embodied emissions per year
fig = px.bar(df_byStatus, x='year', y='emissions_embodied_tons', color = 'status', title='Embodied emissions per year for housing in NL', 
             labels={'emissions_embodied_tons': 'Embodied emissions (tCO2eq)', 'year': 'Year', 'status': 'Emission Type'})
fig.update_layout(width=800)
fig.show()